# Imports

In [9]:
import numpy as np
import pandas as pd

Initialize functions

In [10]:
def categorize(df, categories):
    for category in categories:
        df[category] = pd.Categorical(df[category]).codes
    return df

def numberize(df, numerics):
    for numeric in numerics:
        df[numeric] = df[numeric].astype("float")
    return df

def clean_year(year):
    return 2024 - year

# Function to clean year_built values
def clean_year_built(year):
    try:
        # Attempt to convert the value to an integer
        year = int(float(year))  # Handles both integer and float strings
        year_str = str(year)

        # Ensure the string is exactly 4 characters long
        if len(year_str) > 4:
            return int(year_str[:4])
        elif len(year_str) < 4:
            return int(year_str.ljust(4, '0'))  # Pads with zeros if less than 4 digits
        else:
            return year
    except (ValueError, TypeError):
        # Return NaN for non-numeric values or None
        return pd.NA
    

load and prepare the data

In [13]:
df = pd.read_csv("flatfox.csv", low_memory=False)
df.drop(df[df["offer_type"] != "RENT"].index, inplace=True)
relevant_cat = ["APARTMENT", "HOUSE"]
df.drop(df[~df["object_category"].isin(relevant_cat)].index, inplace=True)
# remove columns with wrong year
df.drop(df[df["year_built"] < 1800].index, inplace=True)
df.drop(df[df["year_built"] > 2030].index, inplace=True)
# NaN  in floors is replaced by 0
# year_built has unrealistic values (for example 19 Million)
#df['year_built'] = df['year_built'].apply(clean_year_built)

df.fillna({"year_built": df["year_built"].mean(), "floor": 0.0, "year_renovated": df["year_built"], "number_of_rooms": 0.0, "livingspace": 0.0}, inplace=True)

df.loc[df["year_renovated"] < 1800, "year_renovated"] = df["year_built"]

df["year_renovated"] = df["year_renovated"].apply(clean_year)
df["year_built"] = df["year_built"].apply(clean_year)

df.dropna(subset=["price_display"], inplace=True)

df.drop([
    "pk",
    "slug",
    "url",
    "short_url",
    "moving_date",
    "rent_net",
    "rent_charges",
    "rent_gross",
    "offer_type",
    "submit_url",
    "status",
    "created",
    "reference",
    "ref_property",
    "ref_house",
    "ref_object",
    "alternative_reference",
    "published",
    "short_title",
    "public_title",
    "pitch_title",
    "description_title",
    "description",
    "attributes",
    "public_address",
    "video_url",
    "tour_url",
    "website_url",
    "live_viewing_url",
    "cover_image",
    "images",
    "documents",
    "agency",
    "rent_title",
    "surface_property",
    "surface_living",
    "surface_usable",
    "surface_usable_minimum",
    "volume",
    "space_display",
    "street"], axis=1, inplace=True)

categories = [
    "object_category",
    "object_type",
    "price_display_type",
    "price_unit",
    "city",
    "moving_date_type"
]
numerics = [
    "is_furnished",
    "is_temporary",
    "is_selling_furniture",
    "reserved"
]

categorize(df, categories)
numberize(df, numerics)
df.describe()
#df.head()
#df.isna().sum()
#df.dtypes

This bright and modern studio ensures maximum comfort in a single room. Modern PU flooring, designer furniture and subdued color combinations give the house an elegant touch. The open living and sleeping area provides a comfortable double bed, wardrobe, flat screen TV and a working area. A fully fitted kitchen caters for all the daily needs: a glass ceramic stove, microwave with baking and grilling options, coffee maker, as well as cutlery and crockery. The separate bathroom with a bathtub is equipped with hand towels, a hair dryer and company-branded toiletries. An extra feature of selected apartments is a balcony.  
  
The Tour Shanelle is located in the heart of Lausanne, directly in the historic Rue Caroline. The downtown can be easily reached from here on foot. Direct neighborhood boasts numerous shopping opportunities, cafés, restaurants and bars. The building with elevator is located between two metro stations, Bessieres and Ours, and is therefore perfectly connected with the pu

,object_category,object_type,price_display,price_display_type,price_unit,number_of_rooms,floor,is_furnished,is_temporary,is_selling_furniture,zipcode,city,latitude,longitude,year_built,year_renovated,moving_date_type,reserved,livingspace
count,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000,11724.000000
mean,0.028233,1.771836,2122.884766,0.999829,0.000171,3.262410,1.990020,0.164961,0.072501,0.027721,5372.672637,923.062692,47.147571,7.993014,39.471739,29.888156,1.016888,0.000597,73.392613
std,0.165644,3.899079,1087.901432,0.013060,0.013060,1.322995,2.159625,0.371161,0.259326,0.164179,2696.155419,576.256038,0.413635,0.841675,23.461238,20.711948,0.727863,0.024429,48.428886
min,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,-5.000000,0.000000,0.000000,0.000000,1000.000000,0.000000,45.826182,5.991881,-1.000000,-1.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1480.000000,1.000000,0.000000,2.500000,1.000000,0.000000,0.000000,0.000000,3072.000000,412.000000,46.950762,7.445634,39.479863,9.000000,0.000000,0.000000,47.000000
50%,0.000000,0.000000,1870.000000,1.000000,0.000000,3.500000,2.000000,0.000000,0.000000,0.000000,4833.000000,892.000000,47.318822,7.909561,39.479863,39.479863,1.000000,0.000000,74.000000
75%,0.000000,0.000000,2500.000000,1.000000,0.000000,4.500000,3.000000,0.000000,0.000000,0.000000,8052.000000,1422.000000,47.434064,8.573304,39.479863,39.479863,2.000000,0.000000,97.000000
max,1.000000,19.000000,15000.000000,1.000000,1.000000,10.500000,31.000000,1.000000,1.000000,1.000000,9657.000000,1812.000000,47.768032,10.364311,224.000000,224.000000,2.000000,1.000000,1275.000000


count all years: 21040

count 1500-2024: 20725

count 1800-2024: 20672


Write clean data to csv

In [24]:
df.to_csv("flatfox_no_parking_clean.csv", index=False)

Create profiling report

In [ ]:
from ydata_profiling import ProfileReport
ProfileReport(pd.read_csv("flatfox.csv", low_memory=False).drop(["created", "published"], axis=1), title="Flatfox raw Profiling").to_file("flatfox_profiling.html")
ProfileReport(pd.read_csv("flatfox_clean.csv", low_memory=False), title="Flatfox clean Profiling").to_file("flatfox_clean_profiling.html")
ProfileReport(pd.read_csv("flatfox_no_parking_clean.csv", low_memory=False), title="Flatfox clean apartment Profiling").to_file("flatfox_no_parking_clean_profiling.html")

Get descriptions

In [7]:
df = pd.read_csv("flatfox.csv", low_memory=False)
descriptions = df["description"]
descriptions.describe()

count                                                 26714
unique                                                25585
top       All Inclusive \n\n\- komplett möbliert (kann e...
freq                                                     52
Name: description, dtype: object